In [27]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants

import numpy as np
from context_nn import ContextNN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[0 1 0 1 0 1 0 1 0 1]


In [94]:
n_input_bits = 10
n_output_bits = 8
n_points = 500
n_point_bits = 5
n_cluster_create_bits = 3
n_cluster_activate_bits = 2

In [259]:
n_input_bits = 256
n_output_bits = 8
n_points = 1000
n_point_bits = 32
n_cluster_create_bits = 6
n_cluster_activate_bits = 4

In [68]:
vector = inp = np.zeros(n_input_bits, dtype=np.int8)
bits = np.random.choice(n_input_bits, n_input_bits // 2, replace=False)
vector[bits] = 1
print(vector)

[0 1 0 0 1 1 1 0 1 0]


In [64]:
print(in_data[0])

[1 0 0 1 1 0 1 0 1 0]


In [279]:
cxnn = context_nn.ContextNN(input_bit_count=n_input_bits,
                            output_bit_count=n_output_bits,
                            watch_point_count=n_points,
                            watch_bit_count=n_point_bits,
                            cluster_make_threshold=n_cluster_create_bits,
                            cluster_activate_threshold=n_cluster_activate_bits)

In [70]:
cxnn.receive_bits(vector, {0})

[0 1 3 4 6]
[2 4 5 8 9]
[1 2 4 6 9]


In [71]:
print([idx for idx, (bit, clusters) in enumerate(cxnn.point_stats()) if clusters > 0])

[32, 36, 38]


In [76]:
point_idx = 38
point = cxnn.point_objects[point_idx]
print(point.watch_bits)
print(point.cluster_objects[0].stats)


[1 2 4 6 9]
{b'\x01\x00\x01\x01\x00': 1}


Random data vectors

In [280]:
def gen_input_output(count=1000, n_in_active_bits=40, n_out_active_bits=4) -> (np.array, np.array):
    inp = np.zeros(shape=(count, n_input_bits), dtype=np.int8)
    out = [] # np.zeros(shape=(count, n_output_bits), dtype=np.int8)
    for i in range(count):
        in_bits = np.random.choice(n_input_bits, n_in_active_bits, replace=False)
        out_bits = np.random.choice(n_output_bits, n_out_active_bits, replace=False)
        inp[i][in_bits] = 1
        out.append(set(out_bits))
    return inp, out

def gen_repeated_vectors(n_base=5, count=1000, n_in_active_bits=40, n_out_active_bits=4) -> (np.array, list):
    in_base, out_base = gen_input_output(n_base, n_in_active_bits, n_out_active_bits)
    in_res = np.zeros(shape=(count, n_input_bits), dtype=np.int8)
    out_res = []
    for i in range(count):
        rnd_idx = np.random.randint(n_base)
        in_res[i, :] = in_base[rnd_idx, :]
        out_res.append(out_base[rnd_idx])
    return in_res, out_res    
    

# in_data, out_data = gen_input_output(count=1000, n_in_active_bits=n_input_bits // 4)
in_data, out_data = gen_repeated_vectors(n_base=1000, count=1000, n_in_active_bits=n_input_bits // 4)
print(out_data[:10])
print(in_data[1])

[{1, 4, 5, 6}, {0, 2, 3, 7}, {0, 1, 4, 6}, {4, 5, 6, 7}, {1, 2, 4, 6}, {1, 4, 5, 6}, {1, 2, 4, 6}, {0, 1, 3, 6}, {0, 3, 4, 7}, {0, 2, 3, 6}]
[0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0
 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0
 1 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1]


In [281]:
%%time
for idx, vector in enumerate(in_data):
    cxnn.receive_bits(vector, out_data[idx])

Wall time: 36.9 s


In [282]:
print(cxnn.cluster_count())
print(cxnn.point_stats())

272340
[(1, 262), (4, 273), (3, 269), (5, 262), (3, 260), (2, 267), (6, 283), (1, 263), (5, 276), (3, 269), (2, 261), (1, 252), (3, 258), (7, 264), (0, 294), (5, 280), (3, 271), (0, 286), (7, 272), (4, 274), (2, 257), (0, 290), (6, 286), (2, 270), (0, 304), (4, 277), (3, 259), (5, 265), (7, 275), (7, 279), (3, 269), (7, 276), (1, 259), (6, 292), (4, 262), (6, 275), (7, 283), (6, 266), (4, 272), (5, 271), (3, 259), (7, 272), (6, 285), (0, 289), (5, 276), (5, 277), (1, 254), (6, 281), (5, 268), (1, 264), (5, 280), (1, 262), (1, 254), (0, 276), (1, 262), (1, 261), (5, 268), (2, 258), (4, 264), (6, 291), (7, 261), (0, 306), (0, 292), (0, 287), (5, 274), (4, 266), (0, 294), (1, 256), (2, 257), (0, 289), (7, 285), (2, 251), (3, 259), (2, 269), (2, 254), (0, 280), (3, 255), (4, 259), (0, 292), (2, 270), (2, 248), (5, 280), (2, 261), (7, 274), (3, 250), (0, 303), (6, 294), (1, 266), (0, 300), (1, 255), (7, 281), (3, 262), (5, 252), (4, 269), (0, 300), (6, 269), (2, 250), (0, 297), (2, 261), (2

In [283]:
point_idx = 5
cluster_idx = 1
point = cxnn.point_objects[point_idx]
active_bits = {i for i, bit in enumerate(in_data[1]) if bit > 0}
print(point.bit_mask)
print(point.watch_bits)
print(set(point.watch_bits) & active_bits)
print(point.cluster_objects[cluster_idx].bit_mask)
print([idx for idx, bit in enumerate(point.cluster_objects[cluster_idx].bit_mask) if bit > 0])
for key, acts in point.cluster_objects[cluster_idx].stats.items():
    bits = [idx for idx, bit in enumerate(np.fromstring(key, dtype=np.int8)) if bit > 0]  
    print(bits, acts)

[1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[  0   3   9  11  48  54  57  65  73  77  84  85  86  88  98 119 131 132
 134 142 146 152 154 155 172 186 190 195 200 209 223 252]
{65, 98, 195, 9, 11, 86, 152, 154, 190}
[1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0]
[0, 4, 7, 9, 10, 21, 22, 29]
[0, 4, 7, 9, 10, 21, 22, 29] 1
[0, 4, 10, 21, 22] 2
[4, 10, 21, 29] 2
[9, 21, 22, 29] 4
[0, 7, 10, 21] 3
[0, 7, 9, 10] 1
[4, 9, 10, 29] 1
[4, 7, 9, 21, 29] 2
[10, 21, 22, 29] 1
[4, 9, 21, 22] 1
[0, 7, 9, 29] 1
